# Dynamic Pricing for Urban Parking Lots
# Capstone Project - Summer Analytics 2025

In [10]:
!pip install pathway bokeh --quiet
# Import required libraries
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import column
import pathway as pw

In [11]:
# Load dataset
df = pd.read_csv("dataset.csv")
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df = df.rename(columns={'SystemCodeNumber': 'parking_lot_id'})

In [12]:
# Constants
BASE_PRICE = 10
ALPHA = 2.0  # for Model 1

# --- Model 1: Baseline Linear Pricing ---
def baseline_linear_price(prev_price, Occupancy, Capacity): # Corrected capitalization
    return prev_price + ALPHA * (Occupancy / Capacity)

# Simulate Model 1
price_history_1 = []
prices = {lot_id: BASE_PRICE for lot_id in df['parking_lot_id'].unique()}
for t in sorted(df['Timestamp'].unique()): # Changed 'time' to 'Timestamp'
    current_df = df[df['Timestamp'] == t] # Changed 'time' to 'Timestamp'
    new_prices = {}
    for _, row in current_df.iterrows():
        lot_id = row['parking_lot_id']
        prev_price = prices.get(lot_id, BASE_PRICE) # Use .get with a default
        price = baseline_linear_price(prev_price, row['Occupancy'], row['Capacity'])
        new_prices[lot_id] = price
        price_history_1.append({'time': t, 'parking_lot_id': lot_id, 'price': price})
    prices = new_prices

# Visualize Model 1
price_df1 = pd.DataFrame(price_history_1)
plots = []
for lot_id in price_df1['parking_lot_id'].unique():
    lot_data = price_df1[price_df1['parking_lot_id'] == lot_id]
    source = ColumnDataSource(lot_data)
    p = figure(title=f"Model 1 - Lot {lot_id} Price Over Time", x_axis_type="datetime", width=800, height=250)
    p.line(x='time', y='price', source=source, line_width=2)
    plots.append(p)
show(column(*plots))

In [13]:
# --- Model 2: Demand-Based Pricing ---
def get_vehicle_weight(vehicle_type):
    return {'car': 1, 'bike': 0.5, 'truck': 1.5}.get(vehicle_type, 1)

def demand_function(row):
    alpha, beta, gamma, delta, epsilon = 1.2, 0.5, 0.3, 2.0, 1.0
    demand = (
        alpha * (row['Occupancy'] / row['Capacity']) +
        beta * row['QueueLength'] -
        gamma * row['TrafficConditionNearby'] +
        delta * row['IsSpecialDay'] +
        epsilon * get_vehicle_weight(row['VehicleType'])
    )
    return demand

def demand_based_price(demand):
    demand = np.clip(demand, 0, 10)
    price = BASE_PRICE * (1 + 0.1 * (demand / 10))
    return np.clip(price, 5, 20)

# Simulate Model 2
price_history_2 = []
prices = {lot_id: BASE_PRICE for lot_id in df['parking_lot_id'].unique()}

# Map categorical fields - TrafficConditionNearby to numeric level
traffic_level_map = {'low': 1, 'average': 2, 'high': 3}


for t in sorted(df['Timestamp'].unique()):
    current_df = df[df['Timestamp'] == t].copy() # Create a copy to avoid SettingWithCopyWarning
    current_df['TrafficConditionNearby'] = current_df['TrafficConditionNearby'].map(traffic_level_map)
    # Handle potential missing values after mapping
    current_df['TrafficConditionNearby'] = current_df['TrafficConditionNearby'].fillna(current_df['TrafficConditionNearby'].median())


    new_prices = {}
    for _, row in current_df.iterrows():
        lot_id = row['parking_lot_id']
        prev_price = prices.get(lot_id, BASE_PRICE) # Use .get with a default
        demand = demand_function(row)
        price = demand_based_price(demand)
        new_prices[lot_id] = price
        price_history_2.append({'time': t, 'parking_lot_id': lot_id, 'price': price})
    prices = new_prices

# Visualize Model 2
price_df2 = pd.DataFrame(price_history_2)
plots = []
for lot_id in price_df2['parking_lot_id'].unique():
    lot_data = price_df2[price_df2['parking_lot_id'] == lot_id]
    source = ColumnDataSource(lot_data)
    p = figure(title=f"Model 2 - Lot {lot_id} Price Over Time", x_axis_type="datetime", width=800, height=250)
    p.line(x='time', y='price', source=source, line_width=2)
    plots.append(p)
show(column(*plots))

In [14]:
# --- Model 3: Competitive Pricing ---
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371
    return c * r

def find_nearest_lot(row, current_prices):
    dists = df[df['parking_lot_id'] != row['parking_lot_id']].groupby('parking_lot_id').first().reset_index()
    dists['distance'] = dists.apply(lambda r: haversine(
        row['Longitude'], row['Latitude'], r['Longitude'], r['Latitude']), axis=1)
    nearest = dists.sort_values('distance').iloc[0]
    return nearest['parking_lot_id'], current_prices.get(nearest['parking_lot_id'], BASE_PRICE)

def competitive_price(row, current_prices):
    base = demand_based_price(demand_function(row))
    lot_id = row['parking_lot_id']
    price = base
    if row['Occupancy'] >= row['Capacity']:
        nearest_id, nearest_price = find_nearest_lot(row, current_prices)
        if nearest_price < base:
            price = base - 1.0
    return max(5, min(price, 20))

# Simulate Model 3
price_history_3 = []
prices = {lot_id: BASE_PRICE for lot_id in df['parking_lot_id'].unique()}

# Map categorical fields - TrafficConditionNearby to numeric level
traffic_level_map = {'low': 1, 'average': 2, 'high': 3}


for t in sorted(df['Timestamp'].unique()):
    current_df = df[df['Timestamp'] == t].copy() # Create a copy to avoid SettingWithCopyWarning
    # Map categorical fields - TrafficConditionNearby to numeric level
    current_df['TrafficConditionNearby'] = current_df['TrafficConditionNearby'].map(traffic_level_map)
    # Handle potential missing values after mapping
    current_df['TrafficConditionNearby'] = current_df['TrafficConditionNearby'].fillna(current_df['TrafficConditionNearby'].median())


    new_prices = {}
    for _, row in current_df.iterrows():
        lot_id = row['parking_lot_id']
        price = competitive_price(row, prices)
        new_prices[lot_id] = price
        price_history_3.append({'time': t, 'parking_lot_id': lot_id, 'price': price})
    prices = new_prices

# Visualize Model 3
price_df3 = pd.DataFrame(price_history_3)
plots = []
# Plot for a few parking lots to avoid overwhelming the output
for i, lot_id in enumerate(price_df3['parking_lot_id'].unique()):
    if i >= 3: # Limit to 3 plots
        break
    lot_data = price_df3[price_df3['parking_lot_id'] == lot_id]
    source = ColumnDataSource(lot_data)
    p = figure(title=f"Model 3 - Lot {lot_id} Price Over Time", x_axis_type="datetime", width=800, height=250)
    p.line(x='time', y='price', source=source, line_width=2)
    plots.append(p)
show(column(*plots))